## Instantiate a small ConvNet

In [1]:
from keras import layers
from keras import models

Using TensorFlow backend.


A ConvNet is basically a stack of Conv2D and MaxPooling2D layers. ConvNet takes input tensors of shape (image_height, image_width, image_channels). We configure it to (28,28,1) for MNIST images.

In [2]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3,3), activation='relu',input_shape=(28,28,1)))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))
model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Conv2D(64, (3,3), activation='relu'))

## View ConvNet Architecture

In [3]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 26, 26, 32)        320       
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 13, 13, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 11, 11, 64)        18496     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 5, 5, 64)          0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 3, 3, 64)          36928     
Total params: 55,744
Trainable params: 55,744
Non-trainable params: 0
_________________________________________________________________


The output of every Conv2D and MaxPooling2D layer is a 3D tensor of shape (height, width, channels). As we move deeper into the network, the width and height dimensions seem to shrink.

The number of channels is controlled by the first argument passed to the Conv2D layers (32 or 64).

## Add a Classifier on top if the ConvNet